In [3]:
import ast
import fileinput
import json
import pickle
import re

from collections import Counter, defaultdict

In [11]:
with open('data/venue_ids.pkl', 'rb') as f:
    confjourdict = pickle.load(f)

In [12]:
confjourdict.keys()

dict_keys(['max_counter', 'confs', 'jours', 'acronyms'])

### Identify paper nodes from A/A* conferenes

In [13]:
counter = 0

network_dict = {}
edge_dict = {}

# relevant_venue_fields = ['_id', 'raw', 'type', 'name_d', 'sid', 't', 'name_s', 'name', 'src']
# venue_stats = {x: Counter() for x in relevant_venue_fields}

with fileinput.input("/home/singh_shruti/data/mag_aminer/citation_network/dblpv13.json") as f:
    for line in f:
        if counter == 0:
            counter += 1
            continue
        counter += 1

        if counter%500000 == 0:
            print(counter)
        
        try:
            line = line.strip()
            if line.startswith(",{"):
                line = line[1:]
            try:
                pdict = json.loads(line)
            except:
                pdict = ast.literal_eval(line)


            if "venue" in pdict:
                if "raw" in pdict["venue"]:
                    raw_venue_name = pdict["venue"]["raw"].lower().strip()
                    found = False

                    for conf_k, acrs in confjourdict['acronyms'].items():
                        for aname in acrs:
    #                         re_search = '\b'+aname.lower()+'\b'
                            x = re.search(r'\b'+aname.lower().strip()+r'\b', raw_venue_name)
                            if x:
                                if "_id" in pdict and pdict["_id"]: 
                                    if type(pdict["_id"])==str:
                                        network_dict[pdict["_id"]] = pdict
                                        network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                    else:
                                        print(counter, pdict["_id"])
                                found = True
                                break
                        if found:
                            break
                    if not found:
                        raw_venue_name_without_of_the = raw_venue_name.replace(" of ", " ")
                        raw_venue_name_without_of_the = raw_venue_name_without_of_the.replace(" the ", " ")
                        raw_venue_name_without_of_the = raw_venue_name_without_of_the.replace(" on ", " ")
                        for conf_k, acrs in confjourdict['confs'].items():
                            for aname in acrs:
                                aname_clean = aname.lower().strip().replace(" of ", " ")
                                aname_clean = aname_clean.replace(" the ", " ")
                                aname_clean = aname_clean.replace(" on ", " ")
                                if raw_venue_name_without_of_the.find(aname_clean) > -1:
                                    if "_id" in pdict and pdict["_id"]: 
                                        if type(pdict["_id"])==str:
                                            network_dict[pdict["_id"]] = pdict
                                            network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                        else:
                                            print(counter, pdict["_id"])
                                    found = True
                                    break
                            if found:
                                break

    #                     if k in relevant_venue_fields:
    #                         venue_stats[k].update([pdict["venue"][k]])
                elif "name_d" in pdict["venue"]:
                    raw_venue_name = pdict["venue"]["name_d"].lower()
                    found = False

                    for conf_k, acrs in confjourdict['jours'].items():
                        for aname in acrs:
                            if raw_venue_name == aname.lower().strip():
                                if "_id" in pdict and pdict["_id"]: 
                                    if type(pdict["_id"])==str:
                                        network_dict[pdict["_id"]] = pdict
                                        network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                    else:
                                        print(counter, pdict["_id"])
                                found = True
                                break
                        if found:
                            break
        except Exception as ex:
            print(counter, ex)

23360 'NoneType' object has no attribute 'lower'
55666 'NoneType' object has no attribute 'lower'
166073 'NoneType' object has no attribute 'lower'
178102 'NoneType' object has no attribute 'lower'
245849 'NoneType' object has no attribute 'lower'
252246 'NoneType' object has no attribute 'lower'
293005 'NoneType' object has no attribute 'lower'
299016 'NoneType' object has no attribute 'lower'
353570 'NoneType' object has no attribute 'lower'
356247 'NoneType' object has no attribute 'lower'
381899 'NoneType' object has no attribute 'lower'
500000
563135 'NoneType' object has no attribute 'lower'
629600 'NoneType' object has no attribute 'lower'
794186 'NoneType' object has no attribute 'lower'
846895 'NoneType' object has no attribute 'lower'
867795 'NoneType' object has no attribute 'lower'
897671 'NoneType' object has no attribute 'lower'
907247 'NoneType' object has no attribute 'lower'
943031 'NoneType' object has no attribute 'lower'
1000000
1029643 'NoneType' object has no attr

3173392 'NoneType' object has no attribute 'lower'
3173669 'NoneType' object has no attribute 'lower'
3173806 'NoneType' object has no attribute 'lower'
3173923 'NoneType' object has no attribute 'lower'
3174603 'NoneType' object has no attribute 'lower'
3174761 'NoneType' object has no attribute 'lower'
3174820 'NoneType' object has no attribute 'lower'
3174945 'NoneType' object has no attribute 'lower'
3175044 'NoneType' object has no attribute 'lower'
3175191 'NoneType' object has no attribute 'lower'
3175192 'NoneType' object has no attribute 'lower'
3175487 'NoneType' object has no attribute 'lower'
3175623 'NoneType' object has no attribute 'lower'
3175687 'NoneType' object has no attribute 'lower'
3175747 'NoneType' object has no attribute 'lower'
3175998 'NoneType' object has no attribute 'lower'
3176345 'NoneType' object has no attribute 'lower'
3176582 'NoneType' object has no attribute 'lower'
3177843 'NoneType' object has no attribute 'lower'
3177975 'NoneType' object has n

3500000
4000000
4500000
5000000


In [14]:
# NEW - papers in A/A* conferences and journals
len(network_dict)

311513

In [46]:
# Count various keys present in the extracted papers

key_counter_across_all_papers = Counter()

for k,v in network_dict.items():
    key_counter_across_all_papers.update(list(v.keys()))

key_counter_across_all_papers

Counter({'_id': 313179,
         'abstract': 263084,
         'authors': 312183,
         'doi': 299178,
         'fos': 298905,
         'isbn': 260172,
         'issn': 253259,
         'issue': 279814,
         'keywords': 210213,
         'lang': 268440,
         'n_citation': 286717,
         'page_end': 307912,
         'page_start': 308503,
         'pdf': 273253,
         'references': 243246,
         'title': 313178,
         'url': 308411,
         'venue': 313179,
         'venue_key_shs': 313179,
         'volume': 280815,
         'year': 313178})

In [17]:
# NEW - venue stats
final_venue_stats = defaultdict(int)

for k, v in network_dict.items():
    final_venue_stats[v['venue_key_shs']] += 1
    
print(sorted(final_venue_stats.items(), key=lambda x: x[1], reverse=True))

[(4, 17787), (43, 16426), (51, 15509), (0, 14849), (6, 12875), (16, 11286), (47, 9658), (2, 9531), (31, 8347), (45, 7490), (33, 7238), (14, 7151), (37, 6572), (57, 6525), (40, 6477), (30, 6125), (21, 6091), (24, 6046), (3, 5757), (46, 5554), (25, 5534), (11, 5392), (1, 5296), (13, 4902), (27, 4853), (52, 4672), (12, 4670), (65, 4514), (19, 3978), (75, 3878), (59, 3144), (83, 2919), (134, 2858), (39, 2836), (18, 2737), (29, 2600), (55, 2550), (58, 2439), (44, 2425), (7, 2416), (8, 2382), (76, 2347), (22, 2330), (26, 2278), (63, 2230), (38, 2063), (32, 2029), (50, 1890), (41, 1740), (71, 1625), (23, 1590), (70, 1545), (82, 1534), (54, 1476), (34, 1463), (78, 1413), (5, 1390), (77, 1375), (28, 1254), (36, 1243), (66, 1240), (49, 1227), (20, 1062), (67, 1023), (10, 1005), (56, 937), (53, 925), (79, 714), (61, 612), (35, 536), (69, 530), (17, 518), (68, 513), (81, 482), (64, 444), (80, 433), (72, 425), (74, 410), (73, 379), (60, 299), (9, 271), (62, 216), (42, 116), (88, 33), (94, 27), (87,

In [45]:
sum(final_venue_stats.values())

311513

In [18]:
conf_papers_count = 0
journal_papers_count = 0
confcount = 0
journcount = 0

for k, v in final_venue_stats.items():
    if k > 83:
        journcount += 1
        journal_papers_count += v
    else:
        confcount += 1
        conf_papers_count += v

In [47]:
#NEW 
conf_papers_count, journal_papers_count

(308563, 2950)

In [48]:
confcount, journcount

(82, 19)

In [20]:
nodes = set()

for k,v in network_dict.items():
    if 'references' in v:
        nodes.update([k])
        for nb in v['references']:
            nodes.update([nb])

In [22]:
edges = defaultdict(list)

for k in nodes:
    if k in network_dict and 'references' in network_dict[k]:
        for i in network_dict[k]['references']:
            edges[k].append(i)

In [23]:
s = 0
for p in edges:
    s += len(edges[p])
    

In [24]:
s

3358659

In [79]:
len(nodes)

745391

# New data stats

In [57]:
import networkx as nx
G = nx.Graph()

In [58]:
G.add_nodes_from(list(nodes))

In [59]:
edges_list = []
for i in edges:
    for k in edges[i]:
        edges_list.append((i, k))

In [60]:
G.add_edges_from(edges_list)

In [61]:
cc = list(nx.connected_components(G))

In [62]:
len_cc = []
for x in cc:
    len_cc.append(len(x))

In [63]:
sum(len_cc)

745391

In [64]:
scc = sorted(len_cc, reverse=True)

In [65]:
len(cc), len(cc[0]), scc[0:5]

(3175, 735735, [735735, 31, 28, 20, 19])

In [66]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 745391\nNumber of edges: 3351090\nAverage degree:   8.9915'

## Directed Graph Statistics

In [67]:
DG = nx.DiGraph(directed=True)

In [68]:
DG.add_nodes_from(list(nodes))

In [69]:
DG.add_edges_from(edges_list)

In [70]:
nx.info(DG)

'Name: \nType: DiGraph\nNumber of nodes: 745391\nNumber of edges: 3354626\nAverage in degree:   4.5005\nAverage out degree:   4.5005'

# Save network dictionary

In [25]:
with open('data/network_dict_june10.pkl', 'wb') as f:
    pickle.dump(network_dict, f)

# Update reference nodes in the network dict

In [26]:
c = 0
missing_ref_nodes = []

for n in nodes:
    if not n in network_dict:
        c+=1
        missing_ref_nodes.append(n)

In [27]:
len(network_dict), len(nodes)

(311513, 745391)

In [28]:
original_network_dict = network_dict.copy()

In [29]:
misingstr = " ".join(missing_ref_nodes)

In [39]:
misingstr.count(" ")

490878

### Add the reference nodes' information to the network dictionary

In [71]:
# %%time
counter = 0

with fileinput.input("/home/singh_shruti/data/mag_aminer/citation_network/dblpv13.json") as f:
    for line in f:
        if counter == 0:
            counter += 1
            continue
        counter += 1

        if counter%100000 == 0:
            print(counter)

        
        try:
            line = line.strip()
            if line.startswith(",{"):
                line = line[1:]
            try:
                pdict = json.loads(line)
            except:
                pdict = ast.literal_eval(line)
            
            found = False
            if "_id" in pdict and pdict["_id"]:
                pid = pdict["_id"]
                if misingstr.find(pid)>-1:
                    
                    if "venue" in pdict:
                        if "raw" in pdict["venue"]:
                            raw_venue_name = pdict["venue"]["raw"].lower().strip()
                            found = False

                            for conf_k, acrs in confjourdict['acronyms'].items():
                                for aname in acrs:
            #                         re_search = '\b'+aname.lower()+'\b'
                                    x = re.search(r'\b'+aname.lower().strip()+r'\b', raw_venue_name)
                                    if x:
                                        if type(pdict["_id"])==str:
                                            network_dict[pdict["_id"]] = pdict
                                            network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                        else:
                                            print(counter, pdict["_id"])
                                        found = True
                                        break
                                if found:
                                    break
                            if not found:
                                raw_venue_name_without_of_the = raw_venue_name.replace(" of ", " ")
                                raw_venue_name_without_of_the = raw_venue_name_without_of_the.replace(" the ", " ")
                                raw_venue_name_without_of_the = raw_venue_name_without_of_the.replace(" on ", " ")
                                for conf_k, acrs in confjourdict['confs'].items():
                                    for aname in acrs:
                                        aname_clean = aname.lower().strip().replace(" of ", " ")
                                        aname_clean = aname_clean.replace(" the ", " ")
                                        aname_clean = aname_clean.replace(" on ", " ")
                                        if raw_venue_name_without_of_the.find(aname_clean) > -1:
                                            if type(pdict["_id"])==str:
                                                network_dict[pdict["_id"]] = pdict
                                                network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                            else:
                                                print(counter, pdict["_id"])
                                            found = True
                                            break
                                    if found:
                                        break


                        if not found and "name_d" in pdict["venue"]:
                            raw_venue_name = pdict["venue"]["name_d"].lower()
                            found = False

                            for conf_k, acrs in confjourdict['jours'].items():
                                for aname in acrs:
                                    if raw_venue_name == aname.lower().strip():
                                        if type(pdict["_id"])==str:
                                            network_dict[pdict["_id"]] = pdict
                                            network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                        else:
                                            print(counter, pdict["_id"])
                                        found = True
                                        break
                                if found:
                                    break
                        if not found:
                            network_dict[pdict["_id"]] = pdict
                            # -1 denoting missing (A/A*) conf/journal name
                            network_dict[pdict["_id"]].update({"venue_key_shs": -1})
                    else:
                        # venue key is missing in the dict (although this is not the case in any entry currently)
                        network_dict[pdict["_id"]] = pdict
                        # -1 denoting missing (A/A*) conf/journal name
                        network_dict[pdict["_id"]].update({"venue_key_shs": -1})
                            
        except Exception as ex:
            print(counter, ex)

2 must be str, not dict
26694 'NoneType' object has no attribute 'lower'
40454 'NoneType' object has no attribute 'lower'
55666 'NoneType' object has no attribute 'lower'
56080 'NoneType' object has no attribute 'lower'
100000
200000
300000
400000
500000
545531 'NoneType' object has no attribute 'lower'
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000


KeyboardInterrupt: 

Update the above code: No need to search for conf name again otherwise these would have been identified in the first run itself.

In [73]:
# %%time
counter = 0

with fileinput.input("/home/singh_shruti/data/mag_aminer/citation_network/dblpv13.json") as f:
    for line in f:
        if counter == 0:
            counter += 1
            continue
        counter += 1

        if counter%100000 == 0:
            print(counter)

        
        try:
            line = line.strip()
            if line.startswith(",{"):
                line = line[1:]
            try:
                pdict = json.loads(line)
            except:
                pdict = ast.literal_eval(line)
            
            found = False
            if "_id" in pdict and pdict["_id"]:
                pid = pdict["_id"]
                if misingstr.find(pid)>-1 and not pid in network_dict:
                    network_dict[pdict["_id"]] = pdict
                    # -1 denoting missing (A/A*) conf/journal name
                    network_dict[pdict["_id"]].update({"venue_key_shs": -1})
        except Exception as ex:
            print(counter, ex)

2 must be str, not dict
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4700000
4800000
4900000
5000000
5100000
5200000
5300000


In [74]:
counter

5354310

In [75]:
c = 0
missing_ref_nodes = []

for n in nodes:
    if not n in network_dict:
        c+=1
        missing_ref_nodes.append(n)

In [76]:
len(missing_ref_nodes)

0

In [78]:
len(nodes), len(network_dict)

(745391, 802392)

In [80]:
set(network_dict.keys()).difference(set(nodes)) 

{'5c2348ceda562935fc1d5622',
 '5bdc316717c44a1f58a075c2',
 '599c7e52601a182cd289094a',
 '5c8c73674895d9cbc6056ef7',
 '573698556e3b12023e71cef2',
 '5a73cb3517c44a0b30355278',
 '573698556e3b12023e71ccfe',
 '6027a5eb91e011c54b48b58a',
 '601d1a4991e011d4d940f1b2',
 '573698566e3b12023e71d1ae',
 '5ff7c5f3d4150a363cab304c',
 '5ce2d256ced107d4c64c894f',
 '53e99aedb7602d9702379adb',
 '53e9af53b7602d9703991722',
 '555048cd45ce0a409eb7188d',
 '53e9b010b7602d9703a6b9cf',
 '53e9ba4eb7602d9704667a7c',
 '5c8b2197e1cd8e8e8b57354e',
 '573698556e3b12023e71ca1c',
 '58d82fced649053542fd6bad',
 '5d9b0cb93a55acb03849652a',
 '53e9b444b7602d9703f407d2',
 '53e9b37bb7602d9703e5ba16',
 '5ef876df91e01159418359d6',
 '53e9b740b7602d97042d8293',
 '5e6cacbb91e011436e691f84',
 '53e99b6ab7602d9702410587',
 '53e9b2aab7602d9703d51de9',
 '5b1642388fbcbf6e5a9b580e',
 '573696fc6e3b12023e5f74d3',
 '53e9b699b7602d970420e9fe',
 '53e9b437b7602d9703f309a8',
 '5550447b45ce0a409eb4e244',
 '5ff8844791e011c832676368',
 '53e99fafb760

In [ ]:
with open('data/network_dict_june10.pkl', 'wb') as f:
    pickle.dump(network_dict, f)

In [44]:
!sed -n '7189'p /home/singh_shruti/data/mag_aminer/citation_network/dblpv13.json

In [51]:
pdict = {"_id": "53e9979eb7602d9701f6c8c4", "title": "ENHANCEMENT OF CELL BOUNDARIES IN TRANSMISSION ELECTRON MICROSCOPY IMAGES.", "authors": [{"_id": "5484cbf2dabfaed7b5fa1bff", "name": "Tolga Tasdizen"}, {"_id": "53f3aa5ddabfae4b34af30b8", "name": "Ross Whitaker"}, {"name": "Robert Marc"}, {"_id": "53f44ac9dabfaeee22a08fba", "name": "Bryan Jones"}], "venue": {"_id": "53a72dad20f7420be8c6e63a", "name_d": "International Conference on Image Processing", "type": 0, "raw": "ICIP (2)"}, "year": 2005, "keywords": ["partial differential equation", "transmission electron microscopy", "image reconstruction", "image texture", "biomedical imaging", "image segmentation", "partial differential equations"], "fos": ["Iterative reconstruction", "Computer vision", "Cellular biophysics", "Computer science", "Image texture", "Segmentation", "Transmission electron microscopy", "Optics", "High-resolution transmission electron microscopy", "Image segmentation", "Artificial intelligence", "Computational analysis"], "n_citation": 36, "page_start": "129", "page_end": "132", "lang": "en", "volume": "2", "issue": "", "issn": "1522-4880", "isbn": "", "doi": "10.1109/ICIP.2005.1530008", "pdf": "//static.aminer.org/pdf/PDF/000/323/341/enhancement_of_cell_boundaries_in_transmission_electron_microscopy_images.pdf", "url": ["http://dx.doi.org/10.1109/ICIP.2005.1530008", "http://www.ncbi.nlm.nih.gov/pubmed/19169423?report=xml&format=text"], "abstract": "Transmission electron microscopy (TEM) is an important modality for the analysis of cellular structures in neurobiology. The computational analysis of neurons entail their segmentation and reconstruction from TEM images. This problem is complicated by the heavily textured nature of cellular TEM images and typically low signal-to-noise ratios. In this paper, we propose a new partial differential equation for enhancing the contrast and continuity of cell membranes in TEM images.", "references": ["558a6e2484ae84d265bd8f8f", "557e27d9f6678c77ea220a7f", "53e9a6a6b7602d9702fdd057", "557f0e62d19faf961d16e78d", "53e9bdfdb7602d9704aaebcd", "53e99a04b7602d970224df26", "558aca9de4b031bae1f9b28e", "53e9b607b7602d970415ea23"]}

In [55]:
pdict.keys(), pdict["venue"]

(dict_keys(['_id', 'title', 'authors', 'venue', 'year', 'keywords', 'fos', 'n_citation', 'page_start', 'page_end', 'lang', 'volume', 'issue', 'issn', 'isbn', 'doi', 'pdf', 'url', 'abstract', 'references']),
 {'_id': '53a72dad20f7420be8c6e63a',
  'name_d': 'International Conference on Image Processing',
  'raw': 'ICIP (2)',
  'type': 0})